In [2]:
# Read input data from files
en_text = pd.read_csv('../data/CONcreTEXT_trial_EN.tsv',sep ='\t')['TEXT']
it_text = pd.read_csv('../data/CONcreTEXT_trial_IT.tsv',sep='\t')['TEXT']

In [33]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split

Eng_train, Eng_test = train_test_split(en_text, test_size=0.20,
                                       random_state=101)

Itl_train, Itl_test = train_test_split(it_text, test_size=0.20,
                                       random_state=101)
print(len(en_text))
print(len(Eng_train))
print(len(Eng_test))

100
80
20


In [4]:
Etext_train = ""
Etext_test = ""
Itext_train = ""
Itext_test = ""
Etext = ""
Itext = ""

for i in Eng_train:
    Etext_train+=i.lower()

for i in Itl_train:
    Itext_train+=i.lower()

for i in en_text:
    Etext+=i.lower()    

for i in it_text:
    Itext+=i.lower()
#print(Etext_train)

In [5]:
# Creating training data and vocab for bigram model
train_bge, vocab_bge = padded_everygram_pipeline(2, Etext_train)
train_bgi, vocab_bgi = padded_everygram_pipeline(2, Itext_train)

In [6]:
# In my solution I converting all the words into lower case to maintain consistency
tokenizer = nltk.RegexpTokenizer(r"\w+")

en_train = []
en_words = []
it_train = []
it_words = []
Efull = []
Ifull = []

for sent in Eng_train:
    for w in word_tokenize(sent):
        en_train.append(w)

for sent in Eng_test:
    for w in word_tokenize(sent):
        en_words.append(w)
        
for sent in Itl_train:
    for w in word_tokenize(sent):
        it_train.append(w)
                
for sent in Itl_test:
    for w in word_tokenize(sent):
        it_words.append(w)
        
for sent in en_text:
    for w in word_tokenize(sent):
        Efull.append(w)
        
for sent in it_text:
    for w in word_tokenize(sent):
        Ifull.append(w)
        
print(u'# English words: {} \t # Italian Words: {}'.format(len(en_words),len(it_words)))

# English words: 336 	 # Italian Words: 288


In [7]:
# Creating training data and vocab for bigram model
train_toke, vocab_toke = padded_everygram_pipeline(2, en_train)
train_toki, vocab_toki = padded_everygram_pipeline(2, it_train)

Etrain_inter, Evocab_inter =  padded_everygram_pipeline(2, en_train)
Itrain_inter, Ivocab_inter =  padded_everygram_pipeline(2, it_train)

In [8]:
#print(Etext_train)

In [9]:
# Building laplace model for bigram
n = 2
lpc_eng = Laplace(n)
lpc_itl = Laplace(n)

In [10]:
# Text model fit
lpc_eng.fit(train_bge, vocab_bge)
lpc_itl.fit(train_bgi, vocab_bgi)

In [11]:
# Tokenize model fit
lpc_eng.fit(train_toke, vocab_toke)
lpc_itl.fit(train_toki, vocab_toki)

In [12]:
# Testing model for English language 
eng_cor = 0
itl_cor = 0
for words in  en_words:
    pbl_eng = 1
    pbl_itl = 1
    for i in range(len(words)):
        if(i< len(words)-1):
            pbl_eng = pbl_eng * lpc_eng.score(words[i+1],[words[i]])
            
            pbl_itl = pbl_itl * lpc_itl.score(words[i+1],[words[i]])
    
    if(pbl_eng > pbl_itl):
        eng_cor = eng_cor + 1
    else:
        eng_cor = eng_cor
    
print("Accuracy of Laplace model for identifying English language in model is :: ", eng_cor/len(en_words))

Accuracy of Laplace model for identifying English language in model is ::  0.6130952380952381


In [13]:
# Testing model for Italian language
eng_cor = 0
itl_cor = 0
for words in  it_words:
    pbl_eng = 1
    pbl_itl = 1
    for i in range(len(words)):
        if(i< len(words)-1):
            pbl_eng = pbl_eng * lpc_eng.score(words[i+1],[words[i]])
            pbl_itl = pbl_itl * lpc_itl.score(words[i+1],[words[i]])
            
#        print("Word is :: ",words)
#        print("Prob of eng",pbl_eng)
#        print("Prob of itl",pbl_itl)
    
    if(pbl_eng <= pbl_itl):
        itl_cor = itl_cor + 1
    else:
        itl_cor = itl_cor
    
print("Accuracy of Laplace model for identifying italian language in model is :: ", itl_cor/len(it_words))

Accuracy of Laplace model for identifying italian language in model is ::  0.875


In [14]:
# Print score for unkown elements in Laplace function for english language
print("Score of unknow characters in italian :: ", lpc_eng.score('!!', ['<UNK>']))
print("Score of unknow characters in italian :: ", lpc_eng.score('-', ['<UNK>'])) 

Score of unknow characters in italian ::  0.008547008547008548
Score of unknow characters in italian ::  0.008547008547008548


In [15]:
# Print score for unkown elements in Laplace function for italian language
print("Score of unknow characters in English :: ", lpc_itl.score('!!', ['<UNK>']))
print("Score of unknow characters in English :: ", lpc_itl.score('-', ['<UNK>']))

Score of unknow characters in English ::  0.008264462809917356
Score of unknow characters in English ::  0.008264462809917356


In [16]:
# Building Interpolated Language Model 
n = 2 # number_of_grams

train_bge, vocab_bge = padded_everygram_pipeline(2, Etext_train)

ipl_eng = KneserNeyInterpolated(n) 
ipl_eng.fit(train_bge, vocab_bge)

Etrain_inter, Evocab_inter =  padded_everygram_pipeline(2, en_train)
ipl_itl = KneserNeyInterpolated(n) 
ipl_itl.fit(Etrain_inter, Evocab_inter)


In [17]:
# Print score for unkown elements in Interpolate function for english language
print("Score of unknow characters in italian :: ", ipl_eng.score('!!', ['<UNK>']))
print("Score of unknow characters in italian :: ", ipl_eng.score('-', ['<UNK>'])) 

Score of unknow characters in italian ::  0.02564102564102564
Score of unknow characters in italian ::  0.02564102564102564


In [18]:
# Print score for unkown elements in Interpolate function for italian language
print("Score of unknow characters in English :: ", ipl_itl.score('!!', ['<UNK>']))
print("Score of unknow characters in English :: ", ipl_itl.score('-', ['<UNK>']))

Score of unknow characters in English ::  0.017543859649122806
Score of unknow characters in English ::  0.017543859649122806


In [19]:
# With the above results I can conclude that Interpolation is giving scores for the characters that are not present in the training dataset

In [20]:
## Q-2) Text generation 

In [21]:
from collections import *

def train_char_lm(fname, order=4):
    data = fname
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in range(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [22]:
# Train bigram model
lm = train_char_lm(Etext_train, order=2)

In [32]:
#print(en_train)

In [24]:
train_toke, vocab_toke = padded_everygram_pipeline(2, en_train)

# Building laplace model for bigram
n = 2
lpc_eng = Laplace(n)

lpc_eng.fit(train_toke, vocab_toke)

In [25]:
print(lpc_eng.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 57 items>


In [26]:
len(lpc_eng.vocab)

57

In [27]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(lpc_eng, num_words,stext, random_seed):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = stext
    for token in lpc_eng.generate(num_words, random_seed):
        if token == '<s>':
            token='~'
        if token == '</s>':
            token='~'
        content.append(token)
    return detokenize(content)

In [386]:
# Bigram model to print sentences with different words
inp_list = {'100' : 'he' , '250': 'ru' , '370': 'fa', '820': 'in', '103': 'yo'}
for i in inp_list:
    vj = generate_sent(lpc_eng, 100,[inp_list[i]], random_seed=i)
    vj1 = vj.replace(' ','')
    vj2 = vj1.replace('~',' ')
    print(vj2)

he  've st  a   eabrgyok  'sercka  andrth  ire ll uf  Fiontainddsth qug inou  dyo ofoublld paren sebes
ru ngs an  ersskste ofendoutincecacte  s dkinsthins hof oth Whas evoug reliurtalle wirour eslpak  ar o
fa , tandoad ed e ive foon   .  off relat ity  teres fonge y   is aioure enstr caintheaskedat r vis fo
in blore  dd bit l n ssng   wil t  Por toan ont her  mer ie  y f gann  d ru esutangr  opla  Whe thess 
yo0  ub ooroisereclof u 'sarbl   , imat  d  Whecererinve  atath  andsind  Pande ct e  theghin  we ia t


In [28]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train_tgi, vocab_tri = padded_everygram_pipeline(3, Efull)

# Building laplace model for bigram
Eng_lap_trigram = Laplace(3)

Eng_lap_trigram.fit(train_tgi, vocab_tri)
print(Eng_lap_trigram.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 58 items>


In [30]:
generate_sent(Eng_lap_trigram, 100,['hel'], '59')

'hel s ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~'

In [73]:
# Trigram model to print sentences with different words
inp_list = {'59' : 'hel' , '19': 'run' , '89': 'fas', '99': 'inv', '69': 'you'}
for i in inp_list:
    vj = generate_sent(Eng_lap_trigram, 100,[inp_list[i]], random_seed=i)
    vj1 = vj.replace(' ','')
    vj2 = vj1.replace('~',' ')
    print(vj2)

hel  Nobods                                                                                            
run Takien                                                                                             
fas des                                                                                                
invng                                                                                                  
youd                                                                                                   


In [392]:
# Building laplace model for bigram - Italian language.

train_toki, vocab_toki = padded_everygram_pipeline(2, it_train)

n = 2
lpc_itl = Laplace(n)

lpc_itl.fit(train_toki, vocab_toki)

# Print output sentence for Italian language using bigram model
inp_list = {'100' : 'an' , '250': 'se' , '370': 'ha', '820': 'un', '103': 'ma'}
for i in inp_list:
    vj = generate_sent(lpc_itl, 100,[inp_list[i]], random_seed=i)
    vj1 = vj.replace(' ','')
    vj2 = vj1.replace('~',' ')
    print(vj2)

an  ci gha alosso  mechituciuele  , mpelliorentti i dielaira  .  die ni ntrton norande  ize  pa antuza
se  pembetelo  ra rno ta ancre porome  dato n   limo vvrsfenostape o  stà nericavi areri    sovera ars
ha o le ai vontrtizonondiaiuesomonasimasee  vonnchi loso r  cccegil la ico covvo tttire  l  suno llema
un  spri  basparbi faiù domi  iso ra   osue ale etre toprio iona  rioli  ppenve entotro   pe ve utarra
ma0 tare anitirengarbe lanialla  Tufisa gad r agrare   e  rgna mbosanersi sttri rl etintate ncelle tto


In [404]:
# Train trigram model for Italian text
train_toki, vocab_toki = padded_everygram_pipeline(3, it_train)

# Building laplace model for trigram
n = 3
Itl_lap_trigram = Laplace(n)

Itl_lap_trigram.fit(train_toki, vocab_toki)
print(Itl_lap_trigram.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 58 items>


In [130]:
# Print output sentence for Italian language using trigram model
inp_list = {'100' : 'anc' , '250': 'se' , '370': 'hai', '820': 'un', '103': 'mar'}
for i in inp_list:
    vj = generate_sent(Itl_lap_trigram, 100,[inp_list[i]], random_seed=i)
    vj1 = vj.replace(' ','')
    vj2 = vj1.replace('~',' ')
    print(vj2)

anc Non il la siticampliare . In presa è una . Il moltando cole offrono avano quantima nuti essere le i
se Non darsi dellevata corri . Un effettergere nuti elementi der aiutare . Adultivo ario dei vasi per 
hai Non sarà sono passere danno scente . Ossere lei ripositivameno inizia è il prendente di rispetto lo
un Non di sare che fissatrire sedi anali , mangia è un un convece più temprentra . Ossere è un donne i
mar Non super batta delici piace per tuo che lo scriva da amici bisogno meetingre altrati viene danno a
